In [1]:
from sympy import symbols, simplify, hessian, solveset, S, solve, log, And, Le, Ge, Eq, Lt, Gt, nonlinsolve, latex, log, Wild, expand_log, logcombine, evaluate,oo, limit,ask, Q, floor, ceiling, lambdify, Sum, sign
from IPython.display import display, HTML, Math

In [2]:
delta_b_negative = False
discrete = False
simp = False

In [3]:
assets = ['b', 's']  # buying and selling assets
base_symbols = ['s', 'v', 'b', 'w', 'j', 'e', 'Delta', 'a', 'min'] 
# spot price, virtual liquidity, balance, weight, jump size, exponent, delta, anchor price, amm-price, jump-multiplier

all_symbols = {}

for asset in assets:
    temp_dict = {}
    for base in base_symbols:
        var_name = f"{base}_{asset}"
        if base == 'e':
            symbol_obj = symbols(var_name, integer=True)
        elif base == 'b':
            symbol_obj = symbols(var_name, nonnegative=True, integer=True)
        elif delta_b_negative and var_name == 'Delta_b':
            symbol_obj = symbols(var_name, negative=True, integer=True)
        else:
            symbol_obj = symbols(var_name, positive=True, integer=True)
        temp_dict[var_name] = symbol_obj
        # Define the variable in the global namespace
        globals()[var_name] = symbol_obj
    all_symbols[asset] = temp_dict.values()
all_symbols

{'b': dict_values([s_b, v_b, b_b, w_b, j_b, e_b, Delta_b, a_b, min_b]),
 's': dict_values([s_s, v_s, b_s, w_s, j_s, e_s, Delta_s, a_s, min_s])}

## Equalities

In [4]:
# affine
def deltaBySpot_(asset, s,v, b, w, j, e, Delta, a, min):
  f = (s - (v + b) * w) / w
  if delta_b_negative and asset == 'b':
    f = -f
  if discrete:
    f = ceiling(f)
  if simp:
    f = simplify(f)
  return f

deltaBySpot = {asset: deltaBySpot_(asset, *all_symbols[asset]) for asset in assets}
deltaBySpot['b']

(s_b - w_b*(b_b + v_b))/w_b

In [5]:
# convex
def spotByExp_(s, v, b, w, j, e, Delta, a, min):
  f = a * ((1 + 1/j) ** e)
  if discrete:
    f = floor(f)
  if simp:
    f = simplify(f)
  return f
spotByExp = {asset: spotByExp_(*all_symbols[asset]) for asset in assets}
spotByExp['b']

a_b*(1 + 1/j_b)**e_b

In [6]:
# convex if not delta_b_negative
def deltaByExp_(asset, s, v, b, w, j, e, Delta, a, min):
  f = deltaBySpot[asset].subs(s, spotByExp[asset])
  if simp:
    f = simplify(f)
  return f

deltaByExp = {asset: deltaByExp_(asset, *all_symbols[asset]) for asset in assets}
deltaByExp['b']

(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b

In [7]:
deltaByExp['s']

(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s

## Target function

we want to minimize the effective price, given a set of exponents for the buying and selling asset each.

In [8]:
if delta_b_negative:
  eff = Delta_s / -Delta_b
else:
  eff = Delta_s / Delta_b
eff

Delta_s/Delta_b

In [9]:
effByExps = eff.subs(Delta_s, deltaByExp['s']).subs(Delta_b, deltaByExp['b'])
if simp:
  effByExps = simplify(effByExps)
effByExps

w_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b)))

In [10]:
H = hessian(effByExps, [e_b, e_s])
H

Matrix([
[2*a_b**2*w_b*(1 + 1/j_b)**(2*e_b)*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)**2/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**3) - a_b*w_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)**2/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**2), -a_b*a_s*w_b*(1 + 1/j_b)**e_b*(1 + 1/j_s)**e_s*log(1 + 1/j_b)*log(1 + 1/j_s)/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**2)],
[                                                                                                                                                        -a_b*a_s*w_b*(1 + 1/j_b)**e_b*(1 + 1/j_s)**e_s*log(1 + 1/j_b)*log(1 + 1/j_s)/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**2),                                      a_s*w_b*(1 + 1/j_s)**e_s*log(1 + 1/j_s)**2/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b)))]])

In [11]:
H[0,0]

2*a_b**2*w_b*(1 + 1/j_b)**(2*e_b)*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)**2/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**3) - a_b*w_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)**2/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**2)

## Inequality-constraints

- value in A0 of buying must not exceed that of selling
- the exponents must adhere to their upper (buying) resp. lower (selling) bounds given by our equation
- need to buy and sell minimum amounts
- cannot buy more than the available balance
- cannot sell more than maxSelling
- the spot prices must not exceed maxInteger
- bonus: the total number of multiplications for both exponentiations must not exceed expLimit (TODO)

### value cannot decrease

In [12]:
a0Buying = spotByExp['s'] * deltaByExp['b'] # < 0 if delta_b_negative, > 0 otherwise
if delta_b_negative:
    a0Buying = -a0Buying # > 0
if simp:
  a0Buying = simplify(a0Buying)
a0Buying

a_s*(1 + 1/j_s)**e_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b

In [13]:
a0Selling = spotByExp['b'] * deltaByExp['s'] # > 0
if simp:
  a0Selling = simplify(a0Selling)
a0Selling

a_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s

In [14]:
a0Bound = a0Buying - a0Selling # <=! 0
if simp:
  a0Bound = simplify(a0Bound)
a0Bound

-a_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s + a_s*(1 + 1/j_s)**e_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b

### exponents cannot result in spot prices that are better than the amm-prices

In [15]:
def expBound_(asset, s, v, b, w, j, e, Delta, a, min):
    e_bound = log(w * (v + b) /a, 1 + 1/j) # constant wrt e
    if simp:
        e_bound = simplify(e_bound)
    if asset == 'b':
        f = e - e_bound # <=! 0 (upper bound)
    else:
        f = e_bound - e # <=! 0 (lower bound)
    if simp:
        f = simplify(f)
    return f

expBound = {asset: expBound_(asset, *all_symbols[asset]) for asset in assets}
expBound['b']

e_b - log(w_b*(b_b + v_b)/a_b)/log(1 + 1/j_b)

In [16]:
expBound['s']

-e_s + log(w_s*(b_s + v_s)/a_s)/log(1 + 1/j_s)

### spot prices cannot exceed maxInteger

In [17]:
max_s, I_max = symbols('max_s I_max', positive=True, integer=True)

maxSpotBound_b = spotByExp['b'] - I_max # <=! 0
if simp:
  maxSpotBound_b = simplify(maxSpotBound_b)
maxSpotBound_b

-I_max + a_b*(1 + 1/j_b)**e_b

In [18]:
maxSpotBound_s = spotByExp['s'] - I_max # <=! 0
if simp:
  maxSpotBound_s = simplify(maxSpotBound_s)
maxSpotBound_s

-I_max + a_s*(1 + 1/j_s)**e_s

### need to swap at least minimum amounts

In [19]:
if delta_b_negative:
  minAmntBound_b = min_b + deltaByExp['b'] # <=! 0
else:
  minAmntBound_b = min_b - deltaByExp['b'] # <=! 0
if simp:
  minAmntBound_b = simplify(minAmntBound_b)
minAmntBound_b

min_b - (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b

In [20]:
minAmntBound_s = min_s - deltaByExp['s'] # <=! 0
if simp:
  minAmntBound_s = simplify(minAmntBound_s)
minAmntBound_s

min_s - (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s

### need to swap at most maximum amounts

In [21]:
if delta_b_negative:
  maxAmntBound_b = -deltaByExp['b'] - b_b # <=! 0
else:
  maxAmntBound_b = deltaByExp['b'] - b_b # <=! 0
if simp:
  maxAmntBound_b = simplify(maxAmntBound_b)
maxAmntBound_b

-b_b + (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b

In [22]:
maxAmntBound_s = deltaByExp['s'] - max_s # <=! 0
maxAmntBound_s

-max_s + (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s

### total number of multiplications of both exponentiations cannot exceed expLimit

In [23]:
# expLimit, aka maximum total number of multiplications for both exponents
m_max = symbols('m_max', positive=True, integer=True)

def numMultsLower_(asset, s,v, b, w, j, e, Delta, a, min):
  return floor(log(e, 2))

# the best number of multiplications we can get by increasing e
def bestMultsAhead_(asset, s,v, b, w, j, e, Delta, a, min):
  return ceiling(log(e, 2))

bestMultsAhead = {asset: bestMultsAhead_(asset, *all_symbols[asset]) for asset in assets}
bestMultsAhead['b']

ceiling(log(e_b)/log(2))

In [24]:
# optimistic estimate. This relies on the vague intuition that exponents will be minimal from the other constraints, so best we can do is increase them if this one is violated
optimisticMultsBound = bestMultsAhead['b'] + bestMultsAhead['s'] - m_max # <=! 0
if simp:
  optimisticMultsBound = simplify(optimisticMultsBound)
optimisticMultsBound

-m_max + ceiling(log(e_b)/log(2)) + ceiling(log(e_s)/log(2))

## Lagrangian

In [25]:
# Decision variables
xs = [e_b, e_s]

# Equality-Constraints
hs = [] 

# Inequality-Constraints
gs = [
    expBound['b'],
    expBound['s'],
    maxSpotBound_b,
    maxSpotBound_s,
    minAmntBound_b,
    minAmntBound_s,
    maxAmntBound_b,
    maxAmntBound_s,
    a0Bound,
    # optimisticMultsBound,
] 

# Lagrange multipliers for equality- and inequality-constraints
mus = symbols('mu_1:{}'.format(len(hs)+1))
lambdas = symbols('lambda_1:{}'.format(len(gs)+1), nonnegative=True)

# Objective function
f = effByExps

# Lagrangian
L = f + sum([mu * h for mu, h in zip(mus, hs)]) + sum([lambda_ * g for lambda_, g in zip(lambdas, gs)])
if simp:
    L = simplify(L)
L

lambda_1*(e_b - log(w_b*(b_b + v_b)/a_b)/log(1 + 1/j_b)) + lambda_2*(-e_s + log(w_s*(b_s + v_s)/a_s)/log(1 + 1/j_s)) + lambda_3*(-I_max + a_b*(1 + 1/j_b)**e_b) + lambda_4*(-I_max + a_s*(1 + 1/j_s)**e_s) + lambda_5*(min_b - (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b) + lambda_6*(min_s - (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s) + lambda_7*(-b_b + (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b) + lambda_8*(-max_s + (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s) + lambda_9*(-a_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s + a_s*(1 + 1/j_s)**e_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b) + w_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b)))

In [26]:
# Equations from stationarity

def custom_diff(f, n):
    diff = f.diff(n)
    # diff = floor(f.subs(n, n+1) - f)
    if simp:
        diff = simplify(diff)
    return diff

equations = []
for x in xs:
  dL_dx = custom_diff(L, x)
  equation = Eq(dL_dx, 0)
  if simp: # results in error from the bestMultsAhead-constraint
    equation = simplify(equation)
  display(equation)
  equations.append(equation)

Eq(a_b*lambda_3*(1 + 1/j_b)**e_b*log(1 + 1/j_b) - a_b*lambda_5*(1 + 1/j_b)**e_b*log(1 + 1/j_b)/w_b + a_b*lambda_7*(1 + 1/j_b)**e_b*log(1 + 1/j_b)/w_b - a_b*w_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))**2) + lambda_1 + lambda_9*(a_b*a_s*(1 + 1/j_b)**e_b*(1 + 1/j_s)**e_s*log(1 + 1/j_b)/w_b - a_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))*log(1 + 1/j_b)/w_s), 0)

Eq(a_s*lambda_4*(1 + 1/j_s)**e_s*log(1 + 1/j_s) - a_s*lambda_6*(1 + 1/j_s)**e_s*log(1 + 1/j_s)/w_s + a_s*lambda_8*(1 + 1/j_s)**e_s*log(1 + 1/j_s)/w_s + a_s*w_b*(1 + 1/j_s)**e_s*log(1 + 1/j_s)/(w_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))) - lambda_2 + lambda_9*(-a_b*a_s*(1 + 1/j_b)**e_b*(1 + 1/j_s)**e_s*log(1 + 1/j_s)/w_s + a_s*(1 + 1/j_s)**e_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))*log(1 + 1/j_s)/w_b), 0)

In [27]:
# Primal feasibility conditions for equality-constraints
for h in hs:
    equation = Eq(h, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

In [28]:
# Complementary slackness conditions
for lambda_, g in zip(lambdas, gs):
    equation = Eq(lambda_ * g, 0)
    if simp:
        equation = simplify(equation)
    display(equation)
    equations.append(equation)

Eq(lambda_1*(e_b - log(w_b*(b_b + v_b)/a_b)/log(1 + 1/j_b)), 0)

Eq(lambda_2*(-e_s + log(w_s*(b_s + v_s)/a_s)/log(1 + 1/j_s)), 0)

Eq(lambda_3*(-I_max + a_b*(1 + 1/j_b)**e_b), 0)

Eq(lambda_4*(-I_max + a_s*(1 + 1/j_s)**e_s), 0)

Eq(lambda_5*(min_b - (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b), 0)

Eq(lambda_6*(min_s - (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s), 0)

Eq(lambda_7*(-b_b + (a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b), 0)

Eq(lambda_8*(-max_s + (a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s), 0)

Eq(lambda_9*(-a_b*(1 + 1/j_b)**e_b*(a_s*(1 + 1/j_s)**e_s - w_s*(b_s + v_s))/w_s + a_s*(1 + 1/j_s)**e_s*(a_b*(1 + 1/j_b)**e_b - w_b*(b_b + v_b))/w_b), 0)

In [29]:
def verify_inequality_constraints(solution, vars, gs):
    subs_dict = dict(zip(vars, solution))
    for g in gs:
        if Gt(g.subs(subs_dict), 0) == True:
            return False
    return True

In [30]:
# Solve
vars = xs + list(mus) + list(lambdas)
solutions = nonlinsolve(equations, vars)
len(solutions)

ValueError: list.remove(x): x not in list

In [ ]:
solutions_ = [solution for solution in solutions if verify_inequality_constraints(solution, vars, gs)]
len(solutions_)

In [ ]:
for solution in solutions_:
    for var, expression in zip(vars, solution):
        display(Eq(var, expression))